In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
this_dir = Path(".").resolve()

In [ ]:
months = list()
for path in (this_dir / "1_Data").rglob("*.xlsx"):
    print(f'Reading {path.name}')
    month = pd.read_excel(path, sheet_name='PR', parse_dates=['Fecha/Señal'],  index_col='Fecha/Señal', usecols="A:AC")
    months.append(month)

In [ ]:
pr = pd.concat(months)

In [ ]:
pr = pr.sort_index()

In [ ]:
# I export the consolidated data into a single Excel file
writer = pd.ExcelWriter('TyM_PR_Source_Data.xlsx', engine='xlsxwriter')
pr.to_excel(writer, sheet_name='Source_PR')
writer.save()

In [ ]:
# In order to work with a pivot table, I need to get rid of the dataframe index
pr.reset_index(inplace=True)

In [ ]:
# Names of columns 1 and 2 are integers instead of text strings, that created me problems when pivoting the data;
# I change the type from integers to strings
pr.rename(columns={1:'1', 2:'2'}, inplace=True)

In [ ]:
# Columns that I want to use in the pivot table
values_pivot = ['Irradiación (kWh/m2) Comunicación',
                'Irradiación (Wh/m2) No filtrada',
                '1',
                '2',
                'Irradiación (kWh/m2) (POAi) Filtrado I',
                'Pf\n(kWp)',
                'Energía M Filtrada2 (kWh)',
                'Energía T Filtrada2 (kWh)',
                'HS']

In [ ]:
# First I pivot the data
pivot_pr = pd.pivot_table(pr,
                          index="Fecha/Señal", values=values_pivot,
                          aggfunc={'Irradiación (kWh/m2) Comunicación': np.sum,
                                   'Irradiación (Wh/m2) No filtrada': np.sum,
                                   'Irradiación (kWh/m2) (POAi) Filtrado I': np.sum,
                                   '1': np.sum,
                                   '2': np.sum,
                                   'Pf\n(kWp)': np.mean,
                                   'Energía M Filtrada2 (kWh)': np.sum,
                                   'Energía T Filtrada2 (kWh)': np.sum,
                                   'HS': np.sum})

In [ ]:
# Then I resample the data from five minutes to days ('D') and months ('M')
pr_daily_summary = pivot_pr.resample('D').agg({
                                               'Irradiación (kWh/m2) Comunicación': np.sum,
                                               'Irradiación (Wh/m2) No filtrada': np.sum,
                                               'Irradiación (kWh/m2) (POAi) Filtrado I': np.sum,
                                               '1': np.sum,
                                               '2': np.sum,
                                               'Pf\n(kWp)': np.mean,
                                               'Energía M Filtrada2 (kWh)': np.sum,
                                               'Energía T Filtrada2 (kWh)': np.sum,
                                               'HS': np.sum})

In [ ]:
pr_monthly_summary = pivot_pr.resample('M').agg({
                                               'Irradiación (kWh/m2) Comunicación': np.sum,
                                               'Irradiación (Wh/m2) No filtrada': np.sum,
                                               'Irradiación (kWh/m2) (POAi) Filtrado I': np.sum,
                                               '1': np.sum,
                                               '2': np.sum,
                                               'Pf\n(kWp)': np.mean,
                                               'Energía M Filtrada2 (kWh)': np.sum,
                                               'Energía T Filtrada2 (kWh)': np.sum,
                                               'HS': np.sum})

In [ ]:
# I create a second pivot with fewer columns
values_pivot_2 = ['Irradiación (kWh/m2) Comunicación',
                  'Energía M Filtrada (kWh)',
                  'Energía T Filtrada (kWh)']

In [ ]:
pivot_pr_2 = pd.pivot_table(pr,
                            index="Fecha/Señal", values=values_pivot_2,
                            aggfunc={'Irradiación (kWh/m2) Comunicación': np.sum,
                                     'Energía M Filtrada (kWh)': np.sum,
                                     'Energía T Filtrada (kWh)': np.sum})

In [ ]:
pr_daily_summary_2 = pivot_pr_2.resample('D').agg({
                                               'Irradiación (kWh/m2) Comunicación': np.sum,
                                               'Energía M Filtrada (kWh)': np.sum,
                                               'Energía T Filtrada (kWh)': np.sum})

In [ ]:
pr_monthly_summary_2 = pivot_pr_2.resample('M').agg({
                                               'Irradiación (kWh/m2) Comunicación': np.sum,
                                               'Energía M Filtrada (kWh)': np.sum,
                                               'Energía T Filtrada (kWh)': np.sum})

In [ ]:
writer = pd.ExcelWriter('TyM_Calculo_PR.xlsx', engine='xlsxwriter')
pr_daily_summary.to_excel(writer, sheet_name='PR_Daily_Incidencias')
pr_monthly_summary.to_excel(writer, sheet_name='PR_Monthly_Incidencias')
pr_daily_summary_2.to_excel(writer, sheet_name='PR_Daily_NoIncidencias')
pr_monthly_summary_2.to_excel(writer, sheet_name='PR_Monthly_NoIncidencias')
writer.save()